<a href="https://colab.research.google.com/github/Nandilath/ML/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classify images with MNIST hand written digit dataset  by using VGG16  and  add 5% gaussian noise with dataset and give me code for above stated Recall,Precision,Classification Report  and plot its loss and accuracy curve and confusion matrix 

Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Load and Preprocess the Data

In [18]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


Add 5% Gaussian Noise

In [19]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=False,
    vertical_flip=False,
    validation_split=0.1,
    data_format='channels_last', # specify data format
    preprocessing_function=lambda x: x + np.random.normal(0, 0.05, x.shape)
)

# add an extra dimension to x_train
x_train = np.expand_dims(x_train, axis=-1)

train_generator = datagen.flow(
    x_train, y_train, batch_size=64, subset='training')

val_generator = datagen.flow(
    x_train, y_train, batch_size=64, subset='validation')


Define the Model

In [21]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), padding='same'),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), padding='same'),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), padding='same'),

    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), padding='same'),

    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), padding='same'),

    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(10, activation='softmax')
])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 28, 28, 64)        640       
                                                                 
 conv2d_14 (Conv2D)          (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_15 (Conv2D)          (None, 14, 14, 128)       73856     
                                                                 
 conv2d_16 (Conv2D)          (None, 14, 14, 128)       147584    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                  

Compile and Train the Model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator,
)


Epoch 1/5
816/844 [============================>.] - ETA: 2:50 - loss: 2.3019 - accuracy: 0.1110

Evaluate the Model

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print('Classification Report:')
print(classification_report(y_test_classes, y_pred_classes))

print('Confusion Matrix:')
print(confusion_matrix(y_test_classes, y_pred_classes))


Plot the Loss and Accuracy Curve

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(history.history['loss'], label='Training Loss')
ax1.plot(history.history['val_loss'], label='Validation Loss')
ax1.set_title('Training and Validation Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()

ax2.plot(history.history['accuracy'], label='Training Accuracy')
ax2.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax2.set_title('Training and Validation Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Add Gaussian noise to the images
noise_factor = 0.05
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_test_noisy = X_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)
X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)

# Display a single noisy image
plt.figure(figsize=(2, 2))
plt.imshow(X_train_noisy[0], cmap='gray')
plt.axis('off')
plt.show()

# Display a single noise-free image
plt.figure(figsize=(2, 2))
plt.imshow(X_train[0], cmap='gray')
plt.axis('off')
plt.show()

# Normalize pixel values to range [0, 1]
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
X_train_noisy = X_train_noisy.astype('float32') / 255
X_test_noisy = X_test_noisy.astype('float32') / 255

# One-hot encode the class labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define the CNN model architecture
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the model on noise-free images
history = model.fit(X_train.reshape(-1, 28, 28, 1), y_train, batch_size=128, epochs=10, validation_split=0.1)

# Evaluate the model on noise-free test images
score = model.evaluate(X_test.reshape(-1, 28, 28, 1), y_test, verbose=0)
print(f"Accuracy on noise-free test images: {score[1]*100:.2f}%")

# Train the model on noisy images
history_noisy = model.fit(X_train_noisy.reshape(-1, 28, 28, 1), y_train, batch_size=128, epochs=10, validation_split=0.1)

# Evaluate the model on noisy test images
score_no


In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape the data to (num_samples, height, width, depth)
X_train = np.reshape(X_train, (-1, 28, 28, 1))
X_test = np.reshape(X_test, (-1, 28, 28, 1))

# Add 5% Gaussian noise to the training and test data
noise_factor = 0.05
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_test_noisy = X_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)

# Clip the noisy images to be between 0 and 1
X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)

# Create a VGG16 model
model = Sequential([
    Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_noisy, y_train, epochs=50, batch_size=128, validation_data=(X_test_noisy, y_test))

# Plot the loss and accuracy curves
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()

# Evaluate the model on the test data
y_pred = np.argmax(model.predict(X_test_noisy), axis=-1)
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))
print('Classification Report:')
print(classification_report(y_test, y_pred))


COMPLETE CODE with 5% NOISE

importing the necessary libraries and loading the MNIST dataset:

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, datasets
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()


add 5% Gaussian noise to the images in the training and testing datasets

In [ ]:
# Add 5% Gaussian noise to the images
noise_factor = 0.05
train_images_noisy = train_images + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=train_images.shape)
test_images_noisy = test_images + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=test_images.shape)

# Clip the noisy images to ensure that pixel values are between 0 and 1
train_images_noisy = np.clip(train_images_noisy, 0., 1.)
test_images_noisy = np.clip(test_images_noisy, 0., 1.)


 a single image from the dataset with and without noise:

In [ ]:
# Plot a single clean image
plt.figure()
plt.imshow(train_images[0], cmap='gray')
plt.title('Clean image')
plt.show()

# Plot the same image with added noise
plt.figure()
plt.imshow(train_images_noisy[0], cmap='gray')
plt.title('Noisy image')
plt.show()


define the VGG16 model and train it on both the clean and noisy datasets:

In [ ]:
# Define the VGG16 model
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dense(4096, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model on clean images
history_clean = model.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=20, validation_split=0.1)

# Train the model on noisy images
history_noisy = model.fit(train_images_noisy.reshape(-1, 28, 28, 1), train_labels, epochs=20, validation_split=0.1)


evaluate its performance using the test dataset:

In [ ]:
# Evaluate the model on clean images
test_loss_clean, test_acc_clean = model.evaluate(test_images.reshape(-1, 28, 28, 1), test_labels, verbose=0)
print(f'Clean images - Test loss: {test_loss_clean:.4f}, Test accuracy: {test_acc_clean:.4f}')

# Evaluate the model on noisy images
test_loss_noisy, test_acc_noisy = model.evaluate(test_images_noisy.reshape(-1, 28, 28, 1), test_labels, verbose=0)
print(f'Noisy images - Test loss: {test_loss_noisy:.4f}, Test accuracy: {test_acc_noisy:.4f}')


calculate the precision, recall, and confusion matrix using scikit-learn

In [ ]:
# Make predictions on the test dataset
test_preds = np.argmax(model.predict(test_images_noisy.reshape(-1, 28, 28, 1)), axis=-1)

# Print the classification report and confusion matrix
print(classification_report(test_labels, test_preds))
print(confusion_matrix(test_labels, test_preds))


plot the loss and accuracy curves for both the clean and noisy datasets 

In [ ]:
# Plot the training and validation loss for clean and noisy datasets
plt.plot(history_clean.history['loss'], label='Clean train')
plt.plot(history_clean.history['val_loss'], label='Clean val')
plt.plot(history_noisy.history['loss'], label='Noisy train')
plt.plot(history_noisy.history['val_loss'], label='Noisy val')
plt.title('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

